In [1]:
import argparse
import copy
import glob
import numpy as np
from molSimplifyAD.dbclass_mongo import tmcMongo
from molSimplifyAD.utils.pymongo_tools import connect2db, insert, count_find, convert2dataframe
from molSimplifyAD.ga_tools import get_mulliken

Using TensorFlow backend.


## Connect to DB

In [2]:
from os.path import expanduser
home = expanduser("~")
dbconfig = json.load(open(home + "/.db_config"))

db = connect2db(user=dbconfig['user'], pwd=dbconfig['pwd'],
                host="localhost", port=27017,
                database='tmc', auth=True)

In [3]:
constraints = {"alpha": {"$in": [20.0]},
               "lig6": {"$ne": "x"},
              }
cursor = db.oct.find(constraints,
                     no_cursor_timeout=True).batch_size(10)

In [4]:
df = convert2dataframe(db, collection='oct', constraints=constraints, normalized=True)
len(df)

/home/crduan/Package/AutomaticDesign/molSimplifyAD/utils/pymongo_tools.py:163: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat(frames)


43074

In [5]:
list(np.unique(df.dropna(subset=["doi"])['doi'].values))

[u'10.1021/acs.jctc.9b00057',
 u'10.1021/acs.jpca.7b08750',
 u'10.1021/acs.jpclett.8b00170',
 u'10.1039/C7SC01247K',
 u'10.1039/C9ME00069K',
 u'10.1039/C9SC02298H']

## Update the DOI based on the existing "publication" column

In [6]:
pub_based_dict = {'ChemSci2019_JP': "10.1039/C9SC02298H",
                  "Duan_JCTC_2019": '10.1021/acs.jctc.9b00057',
                  "JP_JPCA_2017": "10.1021/acs.jpca.7b08750",
                  'S Gugler, Mol. Syst. Des. Eng.': "10.1039/C9ME00069K"}

In [7]:
for key in pub_based_dict:
    print(key, pub_based_dict[key])
    results = db.oct.update_many({"publication": key},
                                 {"$set": {"doi": pub_based_dict[key]}})
    print("updated:", results.matched_count)

('JP_JPCA_2017', '10.1021/acs.jpca.7b08750')
('updated:', 1169)
('ChemSci2019_JP', '10.1039/C9SC02298H')
('updated:', 4653)
('Duan_JCTC_2019', '10.1021/acs.jctc.9b00057')
('updated:', 3421)
('S Gugler, Mol. Syst. Des. Eng.', '10.1039/C9ME00069K')
('updated:', 2076)


## Update the DOI based on the existing "tag" column

In [ ]:
tag_base_dict = {"jpcl-related": '10.1021/acs.jpclett.8b00170',
                "chem-sci-mad": '10.1039/C7SC01247K',
                'OHLDB': "10.1039/C9ME00069K"}

In [ ]:
for key in tag_base_dict:
    print(key, tag_base_dict[key])
    results = db.oct.update_many({"tag": key},
                                 {"$set": {"doi": tag_base_dict[key]}})
    print("updated:", results.matched_count)

## Simple verification

In [ ]:
db.oct.find_one({"doi": '10.1021/acs.jpca.7b08750'})['tag']